In [11]:
from IPython.core.display import display, HTML 
display(HTML("<style>.container { width:100% !important; }</style>"))

In [12]:
import importlib # использую для перегрузки библиотек
import time
from IPython.display import clear_output
import random

import pandas as pd
import json
import numpy as np
import artm

import data_preparing
import artm_model
import work_with_model as wwm

importlib.reload(data_preparing)
importlib.reload(artm_model)
importlib.reload(wwm)

importlib.reload(artm)
clear_output(True)

artm.version() 0.10.0


In [13]:
import artm
print('artm.version()', artm.version())

artm.version() 0.10.0


In [14]:
inf = {}
inf['exp_name'] = 'simple'
inf['test_size'] = 0.1
inf['num_collection_passes'] = 15
inf['topic_number'] = 750
inf['data_path'] = "service_cards_tokenised_remont_only"

In [15]:
inf['docs'] = data_preparing.read_and_prepared(inf['data_path'])
inf['path_train'], inf['path_test'] = data_preparing.form_test_train_set(docs=inf['docs'], name=inf['exp_name'], test_size=inf['test_size'])
inf['path_castom'] = data_preparing.form_castom_set(inf['docs'], name=inf['exp_name'])
inf['model'], inf['theta_train'] = artm_model.create_and_learn_PLSA(inf['path_train'], topic_number=inf['topic_number'], num_collection_passes=inf['num_collection_passes'])

inf['theta_test'] = wwm.get_theta_from_vw(inf, inf['path_test'])
inf['theta_castom'] = wwm.get_theta_from_vw(inf, inf['path_castom'])
clear_output(True)

/Users/kozlinskii/Documents/study/mipt/nir/service_cards/data_preparing.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rich_docs['rand'] = rich_docs['card_id'].map(lambda x: random.uniform(0, 1))


In [16]:
wwm.make_docs_dicts(inf)
wwm.form_card2topic(inf)
wwm.get_topic2rubric(inf)
wwm.measure_accuracy_on_test(inf)
print(inf.keys())

41071  /  66104 0.6213088466658598
dict_keys(['exp_name', 'test_size', 'num_collection_passes', 'topic_number', 'data_path', 'docs', 'path_train', 'path_test', 'path_castom', 'model', 'theta_train', 'theta_test', 'theta_castom', 'card2rubric', 'card2name', 'card2text', 'card2topic_castom', 'card2topic_train', 'card2topic_test', 'card2topic', 'topic2rubric'])


In [17]:
wwm.get_answers_on_castom_advanced(inf)

1  
prediction: 	 remont-i-stroitel_stvo/drugoe/restavrazia-vanny 
 настоящий: 	  remont-i-stroitel_stvo/santehniceskie-raboty-i-otoplenie/ 
 name:  установка душевого поддона акрил сталь 
 установка душевого поддона акрил сталь  


2  
prediction: 	 remont-i-stroitel_stvo/master-na-cas/sborka-mebeli 
 настоящий: 	  remont-i-stroitel_stvo/mebel_/ 
 name:  сборка офисной мебели 
 сборка офисной мебели осуществляем сборку и разборку офисной мебели упаковку мебели подготовку мебели к переезду или хранению имеется огромный опыт по переезду офисов по 200 300 раб мест составление плана переезда строгое выполнение сроков проведения работ ремонт бу мебели для офиса хранение излишков офисной мебели на нашем складе 


3  
prediction: 	 remont-i-stroitel_stvo/okna-i-balkony/montaj-otkosov 
 настоящий: 	  remont-i-stroitel_stvo/okna-i-balkony/ 
 name:  очищение рамы очистителем kosmofen 
 очищение рамы очистителем kosmofen 1 створчатое окно 2500 2 створчатое окно 3300 3 створчатое окно 4200 балкон

In [18]:
inf_weigth_01 = {}
inf_weigth_01['exp_name'] = 'weight_1_0_2_3'
inf_weigth_01['test_size'] = 0.1
inf_weigth_01['num_collection_passes'] = 15
inf_weigth_01['topic_number'] = 750
inf_weigth_01['data_path'] = "service_cards_tokenised_remont_only"

inf_weigth_01['docs'] = data_preparing.read_and_prepared(inf_weigth_01['data_path'])
inf_weigth_01['path_train'], inf_weigth_01['path_test'] = data_preparing.form_test_train_set(docs=inf_weigth_01['docs'], name=inf_weigth_01['exp_name'], test_size=inf_weigth_01['test_size'])
inf_weigth_01['path_castom'] = data_preparing.form_castom_set(inf_weigth_01['docs'], name=inf_weigth_01['exp_name'])
inf_weigth_01['model'], inf_weigth_01['theta_train'] = artm_model.create_and_learn_PLSA_class_ids_weigth(
                                                            inf_weigth_01['path_train'], 
                                                            topic_number=inf_weigth_01['topic_number'], 
                                                            num_collection_passes=inf_weigth_01['num_collection_passes'], 
                                                            weigths=[1., 0., 2., 3.]
                                                         )

inf_weigth_01['theta_test'] = wwm.get_theta_from_vw(inf_weigth_01, inf_weigth_01['path_test'])
inf_weigth_01['theta_castom'] = wwm.get_theta_from_vw(inf_weigth_01, inf_weigth_01['path_castom'])
clear_output(True)

wwm.make_docs_dicts(inf_weigth_01)
wwm.form_card2topic(inf_weigth_01)
wwm.get_topic2rubric(inf_weigth_01)
wwm.measure_accuracy_on_test(inf_weigth_01)

42898  /  66104 0.6489471136391142


In [19]:
inf_weigth_02 = {}
inf_weigth_02['exp_name'] = 'weight_1_0_3_10'
inf_weigth_02['test_size'] = 0.1
inf_weigth_02['num_collection_passes'] = 15
inf_weigth_02['topic_number'] = 750
inf_weigth_02['data_path'] = "service_cards_tokenised_remont_only"

inf_weigth_02['docs'] = data_preparing.read_and_prepared(inf_weigth_02['data_path'])
inf_weigth_02['path_train'], inf_weigth_02['path_test'] = data_preparing.form_test_train_set(docs=inf_weigth_02['docs'], name=inf_weigth_02['exp_name'], test_size=inf_weigth_02['test_size'])
inf_weigth_02['path_castom'] = data_preparing.form_castom_set(inf_weigth_02['docs'], name=inf_weigth_02['exp_name'])
inf_weigth_02['model'], inf_weigth_02['theta_train'] = artm_model.create_and_learn_PLSA_class_ids_weigth(
                                                            inf_weigth_02['path_train'], 
                                                            topic_number=inf_weigth_02['topic_number'], 
                                                            num_collection_passes=inf_weigth_02['num_collection_passes'], 
                                                            weigths=[1., 0., 3., 10.]
                                                         )

inf_weigth_02['theta_test'] = wwm.get_theta_from_vw(inf_weigth_02, inf_weigth_02['path_test'])
inf_weigth_02['theta_castom'] = wwm.get_theta_from_vw(inf_weigth_02, inf_weigth_02['path_castom'])
clear_output(True)

wwm.make_docs_dicts(inf_weigth_02)
wwm.form_card2topic(inf_weigth_02)
wwm.get_topic2rubric(inf_weigth_02)
wwm.measure_accuracy_on_test(inf_weigth_02)

47035  /  66104 0.7115303158659083


In [20]:
inf_weigth_03 = {}
inf_weigth_03['exp_name'] = 'weight_1_0_10_50'
inf_weigth_03['test_size'] = 0.1
inf_weigth_03['num_collection_passes'] = 15
inf_weigth_03['topic_number'] = 750
inf_weigth_03['data_path'] = "service_cards_tokenised_remont_only"

inf_weigth_03['docs'] = data_preparing.read_and_prepared(inf_weigth_03['data_path'])
inf_weigth_03['path_train'], inf_weigth_03['path_test'] = data_preparing.form_test_train_set(docs=inf_weigth_03['docs'], name=inf_weigth_03['exp_name'], test_size=inf_weigth_03['test_size'])
inf_weigth_03['path_castom'] = data_preparing.form_castom_set(inf_weigth_03['docs'], name=inf_weigth_03['exp_name'])
inf_weigth_03['model'], inf_weigth_03['theta_train'] = artm_model.create_and_learn_PLSA_class_ids_weigth(
                                                            inf_weigth_03['path_train'], 
                                                            topic_number=inf_weigth_03['topic_number'], 
                                                            num_collection_passes=inf_weigth_03['num_collection_passes'], 
                                                            weigths=[1., 0., 10., 50.]
                                                         )

inf_weigth_03['theta_test'] = wwm.get_theta_from_vw(inf_weigth_03, inf_weigth_03['path_test'])
inf_weigth_03['theta_castom'] = wwm.get_theta_from_vw(inf_weigth_03, inf_weigth_03['path_castom'])
clear_output(True)

wwm.make_docs_dicts(inf_weigth_03)
wwm.form_card2topic(inf_weigth_03)
wwm.get_topic2rubric(inf_weigth_03)
wwm.measure_accuracy_on_test(inf_weigth_03)

50348  /  66104 0.76164831175118


In [21]:
inf_weigth_04 = {}
inf_weigth_04['exp_name'] = 'weight_1_0_50_250'
inf_weigth_04['test_size'] = 0.1
inf_weigth_04['num_collection_passes'] = 15
inf_weigth_04['topic_number'] = 750
inf_weigth_04['data_path'] = "service_cards_tokenised_remont_only"

inf_weigth_04['docs'] = data_preparing.read_and_prepared(inf_weigth_04['data_path'])
inf_weigth_04['path_train'], inf_weigth_04['path_test'] = data_preparing.form_test_train_set(docs=inf_weigth_04['docs'], name=inf_weigth_04['exp_name'], test_size=inf_weigth_04['test_size'])
inf_weigth_04['path_castom'] = data_preparing.form_castom_set(inf_weigth_04['docs'], name=inf_weigth_04['exp_name'])
inf_weigth_04['model'], inf_weigth_04['theta_train'] = artm_model.create_and_learn_PLSA_class_ids_weigth(
                                                            inf_weigth_04['path_train'], 
                                                            topic_number=inf_weigth_04['topic_number'], 
                                                            num_collection_passes=inf_weigth_04['num_collection_passes'], 
                                                            weigths=[1., 0., 50., 250.]
                                                         )

inf_weigth_04['theta_test'] = wwm.get_theta_from_vw(inf_weigth_04, inf_weigth_04['path_test'])
inf_weigth_04['theta_castom'] = wwm.get_theta_from_vw(inf_weigth_04, inf_weigth_04['path_castom'])
clear_output(True)

wwm.make_docs_dicts(inf_weigth_04)
wwm.form_card2topic(inf_weigth_04)
wwm.get_topic2rubric(inf_weigth_04)
wwm.measure_accuracy_on_test(inf_weigth_04)

49072  /  66104 0.7423453951349389


In [22]:
inf_weigth_05 = {}
inf_weigth_05['exp_name'] = 'weight_1_0_50_50'
inf_weigth_05['test_size'] = 0.1
inf_weigth_05['num_collection_passes'] = 15
inf_weigth_05['topic_number'] = 750
inf_weigth_05['data_path'] = "service_cards_tokenised_remont_only"

inf_weigth_05['docs'] = data_preparing.read_and_prepared(inf_weigth_05['data_path'])
inf_weigth_05['path_train'], inf_weigth_05['path_test'] = data_preparing.form_test_train_set(docs=inf_weigth_05['docs'], name=inf_weigth_05['exp_name'], test_size=inf_weigth_05['test_size'])
inf_weigth_05['path_castom'] = data_preparing.form_castom_set(inf_weigth_05['docs'], name=inf_weigth_05['exp_name'])
inf_weigth_05['model'], inf_weigth_05['theta_train'] = artm_model.create_and_learn_PLSA_class_ids_weigth(
                                                            name=inf_weigth_05['path_train'], 
                                                            topic_number=inf_weigth_05['topic_number'], 
                                                            num_collection_passes=inf_weigth_05['num_collection_passes'], 
                                                            weigths=[1.0, 0.0, 50.0, 50.0]
                                                         )

inf_weigth_05['theta_test'] = wwm.get_theta_from_vw(inf_weigth_05, inf_weigth_05['path_test'])
inf_weigth_05['theta_castom'] = wwm.get_theta_from_vw(inf_weigth_05, inf_weigth_05['path_castom'])
clear_output(True)

wwm.make_docs_dicts(inf_weigth_05)
wwm.form_card2topic(inf_weigth_05)
wwm.get_topic2rubric(inf_weigth_05)
wwm.measure_accuracy_on_test(inf_weigth_05)

48544  /  66104 0.7343579813627013


In [ ]:
def get_theta_from_vw_third(inf, path):
    batch_vectorizer = None
    batch_vectorizer = artm.BatchVectorizer(data_path='./' + path,
                                                  data_format='vowpal_wabbit',
                                                  target_folder='folder' + path)
    return inf['model'].transform(batch_vectorizer=batch_vectorizer, predict_class_id='@third')

In [ ]:
theta_castom_third = get_theta_from_vw_third(inf_weigth_04, inf_weigth_04['path_castom'])

In [ ]:
theta_castom_third.shape

In [ ]:
#theta_castom_third = get_theta_from_vw_third(inf_weigth_04, inf_weigth_04['@third'])

In [ ]:
#theta_castom_third.shape